In [1]:
import multiprocessing
import os
from re import A
from typing import ItemsView

from py import log
os.environ['PYTHONDONTWRITEBYTECODE'] = '1'
from xtquant import xttrader
from xtquant import xtdata
from xtquant.xttrader import XtQuantTrader, XtQuantTraderCallback
from xtquant.xttype import StockAccount
from xtquant import xtconstant
import pandas_market_calendars as mcal
import pandas as pd
import akshare as ak

import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.dates as mdates

from IPython.core.display import display, HTML
display(HTML("<style>div.output_scroll { height: auto; }</style>"))
def plot(df, column):
    
    # 设置中文字体
    mpl.rcParams['font.sans-serif'] = ['SimHei']  # 指定中文字体为 SimHei
    mpl.rcParams['axes.unicode_minus'] = False  # 解决负号显示问题

    # 假设这是你的DataFrame，其中包含收益

    # 绘制收益曲线
    # 将日期列转换为日期类型
    # df['date'] = pd.to_datetime(df['date'])

    # # 设置日期列为索引
    # df.set_index('date', inplace=True)
    df.index = pd.to_datetime(df.index)
    # 绘制收益曲线
    plt.figure(figsize=(10, 6))  # 设置图表大小
    plt.plot(df.index, df[column], label='Return')
    plt.title('收益曲线')
    plt.xlabel('日期')
    plt.ylabel('收益率')
    plt.legend()  # 显示图例
    plt.grid(True)  # 显示网格线

    # plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
    # plt.gca().xaxis.set_major_locator(mdates.DayLocator(interval=10))

    plt.show()




def caculate_returns(returns_df, row, _print = False):
    r = {}
    cumulative_returns = (1 + returns_df[row]).cumprod()

    # 计算最大回撤
    cumulative_max = cumulative_returns.cummax()
    drawdown = (cumulative_returns - cumulative_max) / cumulative_max
    max_drawdown = drawdown.min()

    # 计算夏普比率
    risk_free_rate = 0.0  # 假设无风险利率为0
    sharpe_ratio = (returns_df[row].mean() - risk_free_rate) / returns_df[row].std()

    # 计算总收益率
    total_return = cumulative_returns.iloc[-1] - 1

    # 计算波动率
    volatility = returns_df[row].std()

    # 计算总盈亏
    total_profit_loss = cumulative_returns.iloc[-1] - cumulative_returns.iloc[0]

    # 计算成功次数、胜率、平均盈利、平均亏损、最大盈利、最大亏损以及盈亏比
    profitable_trades = returns_df[row][returns_df[row] > 0].count()
    losing_trades = returns_df[row][returns_df[row] < 0].count()
    win_rate = profitable_trades / (profitable_trades + losing_trades) if (profitable_trades + losing_trades) > 0 else 0
    average_profit = returns_df[row][returns_df[row] > 0].mean() if profitable_trades > 0 else 0
    average_loss = returns_df[row][returns_df[row] < 0].mean() if losing_trades > 0 else 0
    max_profit = returns_df[row].max()
    max_loss = returns_df[row].min()
    total_trades = profitable_trades + losing_trades
    total_times = returns_df[row].count()
    trade_pct = total_times / total_trades
    profit_loss_ratio = average_profit / abs(average_loss) if average_loss != 0 else 0
    kelly_fraction = win_rate - ((1 - win_rate) / profit_loss_ratio) if profit_loss_ratio != 0 else 0
    return_per_day = total_return / total_times
    year_return = return_per_day * 240

    r['最大回撤'] =  max_drawdown
    r['夏普比率'] =  sharpe_ratio
    r['总收益率'] =  total_return
    r['波动率'] = volatility
    r['年化收益率'] = year_return
    r['总盈亏'] = total_profit_loss
    r['成功次数'] = profitable_trades
    r['失败次数'] = losing_trades
    r['总天数'] = total_times
    r['总交易次数'] = total_trades
    r['交易频率'] = trade_pct
    r['胜率'] = win_rate
    r['平均盈利'] = average_profit
    r['平均亏损'] = average_loss
    r['最大盈利'] = max_profit
    r['最大亏损'] = max_loss
    r['盈亏比'] = profit_loss_ratio
    r['凯利公式最佳仓位'] = kelly_fraction
    if _print:
        print(f"最大回撤: {max_drawdown:.2%}")
        print(f"夏普比率: {sharpe_ratio:.2f}")
        print(f"总收益率: {total_return:.2%}")
        print(f"年化收益率: {year_return:.2%}")
        print(f"波动率: {volatility:.2%}")
        print(f"总盈亏: {total_profit_loss:.2%}")
        print(f"成功次数: {profitable_trades}")
        print(f"失败次数: {losing_trades}")
        print(f"总天数: {total_times}")
        print(f"总交易次数: {total_trades}")
        print(f"交易频率: {trade_pct:.2%}")
        print(f"胜率: {win_rate:.2%}")
        print(f"平均盈利: {average_profit:.2%}")
        print(f"平均亏损: {average_loss:.2%}")
        print(f"最大盈利: {max_profit:.2%}")
        print(f"最大亏损: {max_loss:.2%}")
        print(f"盈亏比: {profit_loss_ratio:.2f}")
        print(f"凯利公式最佳仓位: {kelly_fraction:.2%}")

    return r

d:\anaconda3\envs\python3.8\lib\site-packages\exchange_calendars\exchange_calendar.py:2345: FutureWarning: 'T' is deprecated and will be removed in a future version. Please use 'min' instead of 'T'.
  align: pd.Timedelta | str = pd.Timedelta(1, "T"),
C:\Users\Administrator\AppData\Local\Temp\ipykernel_21132\2954539298.py:21: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [ ]:
strategy_name = '低吸'

sub_strategy_name = '低位N字低吸'

max_stock_rank = 5

xiaocao_fangxiang_filter = True

# months = [ '202409', '202410', '202411', '202412', '202501', '202502' ]

months = ['202501', '202502' ]

hd_pct = 0.003

def group_filter(group, filtered = True, fx_filtered = True, topn = 3):
    if not filtered:
        valid_rows = group[(group['open_price'] > 0) & (group['next_day_open_price'] > 0) & (group['stock_rank'] <= topn)]
        if len(valid_rows) > 0:
            valid_rows['return'] = valid_rows['next_day_open_price'] / valid_rows['open_price'] - 1
            valid_rows['real_return'] = valid_rows['return'] - hd_pct
            avg_value = valid_rows['return'].mean()
            rank_one_row = group[group['stock_rank'] == 1].copy()
            if len(rank_one_row) > 0:
                # 将平均值赋给 rank 为 1 的行的指定列
                rank_one_row['return'] = avg_value
                rank_one_row['real_return'] = avg_value - hd_pct
                return rank_one_row
        else:
            rank_one_row = group[group['stock_rank'] == 1].copy()
            if len(rank_one_row) > 0:
                rank_one_row['return'] = -1
                rank_one_row['real_return'] = -1
                return rank_one_row
    else:
        if fx_filtered:
            min_category_rank = group['max_block_category_rank'].min()
            if min_category_rank > 2:
                rank_one_row = group[group['stock_rank'] == 1].copy()
                if len(rank_one_row) > 0:
                    rank_one_row['return'] = rank_one_row['next_day_open_price'] / rank_one_row['open_price'] - 1
                    rank_one_row['real_return'] = rank_one_row['return'] - hd_pct
                    return rank_one_row
            else:
                category_filtered = group[group['max_block_category_rank'] <= min_category_rank + 2]
                result = category_filtered[category_filtered['max_block_code_rank'] == category_filtered['max_block_code_rank'].min()]
                if len(result) > 1:
                    result = result[result['stock_rank'] == result['stock_rank'].min()]
                result['return'] = result['next_day_open_price'] / result['open_price'] - 1
                result['real_return'] = result['return'] - hd_pct
                return result
        else:
            rank_one_row = group[group['stock_rank'] == 1].copy()
            if len(rank_one_row) > 0:
                rank_one_row['return'] = rank_one_row['next_day_open_price'] / rank_one_row['open_price'] - 1
                rank_one_row['real_return'] = rank_one_row['return'] - hd_pct
            return rank_one_row

In [3]:
import sqlite3

combined_df = pd.DataFrame()
for month in months:
    conn = sqlite3.connect('D:\workspace\TradeX\ezMoney\sqlite_db\strategy_data.db')
    db_name = 'strategy_data_aftermarket_%s' % month
    query = "select * from %s where (strategy_name = '%s' and sub_strategy_name = '%s' and stock_rank <= %s) " % (db_name, strategy_name, sub_strategy_name, max_stock_rank)
    df = pd.read_sql_query(query, conn)
    combined_df = pd.concat([combined_df, df], axis=0)
combined_df = combined_df.reset_index(drop=True)
combined_df

,id,date_key,strategy_name,sub_strategy_name,stock_code,stock_name,stock_rank,block_category,block_codes,industry_code,...,mod_name,mod_short_line_score,mod_short_line_score_change,mod_short_line_rank,mod_trend_score,mod_trend_score_change,mod_trend_rank,env_json_info,block_category_info,created_at
0,919,2024-09-02,低吸,低位N字低吸,603176.XSHG,汇通集团,1,"000008.BKDL,000010.BKDL,000011.BKDL,000007.BKD...","885514.DDBK,885991.DDBK,885705.DDBK,885690.DDB...",980338.ZHBK,...,低位N字低吸,109.97,8.81,2,107.82,2.64,2,"{""9A0001"": {""realShortLineScore"": 23.04, ""real...","{""000027.BKDL"": {""categoryCode"": ""000027.BKDL""...",2025-02-15 17:42:06
1,920,2024-09-02,低吸,低位N字低吸,002144.XSHE,宏达高科,2,,,,...,低位N字低吸,109.97,8.81,2,107.82,2.64,2,"{""9A0001"": {""realShortLineScore"": 23.04, ""real...","{""000027.BKDL"": {""categoryCode"": ""000027.BKDL""...",2025-02-15 17:42:06
2,947,2024-09-03,低吸,低位N字低吸,002428.XSHE,云南锗业,1,,,,...,低位N字低吸,112.68,2.71,1,108.22,-12.05,1,"{""9A0001"": {""realShortLineScore"": 20.72, ""real...","{""000028.BKDL"": {""categoryCode"": ""000028.BKDL""...",2025-02-15 17:43:09
3,1038,2024-09-05,低吸,低位N字低吸,002383.XSHE,合众思壮,1,"000008.BKDL,000023.BKDL,000019.BKDL,000016.BKD...","886021.DDBK,885574.DDBK,885932.DDBK,885564.DDB...","980363.ZHBK,980366.ZHBK",...,低位N字低吸,86.30,11.16,2,94.02,3.22,2,"{""9A0001"": {""realShortLineScore"": -12.1, ""real...","{""000003.BKDL"": {""categoryCode"": ""000003.BKDL""...",2025-02-15 17:45:19
4,1039,2024-09-05,低吸,低位N字低吸,605588.XSHG,冠石科技,2,"000014.BKDL,000006.BKDL,000031.BKDL,000029.BKDL","885785.DDBK,885806.DDBK,885800.DDBK,885738.DDB...",980357.ZHBK,...,低位N字低吸,86.30,11.16,2,94.02,3.22,2,"{""9A0001"": {""realShortLineScore"": -12.1, ""real...","{""000003.BKDL"": {""categoryCode"": ""000003.BKDL""...",2025-02-15 17:45:19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
228,1709,2025-02-25,低吸,低位N字低吸,601789.XSHG,宁波建工,2,"000003.BKDL,000008.BKDL,000023.BKDL,000011.BKD...","886021.DDBK,885757.DDBK,885957.DDBK,885690.DDB...",980338.ZHBK,...,低位N字低吸,26.80,1.16,38,-3.11,-9.69,33,"{""9A0001"": {""realShortLineScore"": 38.02, ""real...","{""000033.BKDL"": {""categoryCode"": ""000033.BKDL""...",2025-02-26 08:01:18
229,1847,2025-02-26,低吸,低位N字低吸,002883.XSHE,中设股份,1,"000008.BKDL,000024.BKDL,000018.BKDL,000011.BKD...","885993.DDBK,885692.DDBK,886021.DDBK,885962.DDB...","980338.ZHBK,980343.ZHBK",...,低位N字低吸,4.88,-31.83,42,-3.49,-6.46,40,"{""9A0001"": {""realShortLineScore"": 36.86, ""real...","{""000027.BKDL"": {""categoryCode"": ""000027.BKDL""...",2025-02-27 08:01:10
230,1848,2025-02-26,低吸,低位N字低吸,603789.XSHG,星光农机,2,"000003.BKDL,000024.BKDL,000023.BKDL,000010.BKD...","885924.DDBK,885739.DDBK,885706.DDBK,885517.DDB...",980365.ZHBK,...,低位N字低吸,4.88,-31.83,42,-3.49,-6.46,40,"{""9A0001"": {""realShortLineScore"": 36.86, ""real...","{""000027.BKDL"": {""categoryCode"": ""000027.BKDL""...",2025-02-27 08:01:10
231,2057,2025-02-28,低吸,低位N字低吸,002209.XSHE,达 意 隆,1,"000023.BKDL,000011.BKDL,000022.BKDL,000033.BKDL","885869.DDBK,885517.DDBK,885882.DDBK,886057.DDB...",980365.ZHBK,...,低位N字低吸,22.14,27.10,31,13.46,12.20,25,"{""9C0001"": {""realShortLineScore"": -14.24, ""rea...","{""000033.BKDL"": {""categoryCode"": ""000033.BKDL""...",2025-02-28 08:01:58


In [4]:
df1 = combined_df.groupby(['date_key', 'strategy_name', 'sub_strategy_name']).apply(group_filter, filtered = True, fx_filtered = False).reset_index(drop=True)
df2 = combined_df.groupby(['date_key','strategy_name','sub_strategy_name']).apply(group_filter, filtered = False, fx_filtered = True).reset_index(drop=True)
df3 = combined_df.groupby(['date_key','strategy_name','sub_strategy_name']).apply(group_filter, filtered = True, fx_filtered = True).reset_index(drop=True)
df4 = combined_df.groupby(['date_key','strategy_name','sub_strategy_name']).apply(group_filter, filtered = False, fx_filtered = True, topn = 2).reset_index(drop=True)
df5 = combined_df.groupby(['date_key','strategy_name','sub_strategy_name']).apply(group_filter, filtered = False, fx_filtered = True, topn = 4).reset_index(drop=True)

m = {
    # 过滤top1
    'rank1': df1,
    # top3
    'rank3': df2,
    # 方向
    'fx1': df3,
    'rank2': df4,
    'rank4': df5,
}

C:\Users\Administrator\AppData\Local\Temp\ipykernel_21132\4003466928.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df1 = combined_df.groupby(['date_key', 'strategy_name', 'sub_strategy_name']).apply(group_filter, filtered = True, fx_filtered = False).reset_index(drop=True)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_21132\4003466928.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df2 = com

In [5]:
df3

,id,date_key,strategy_name,sub_strategy_name,stock_code,stock_name,stock_rank,block_category,block_codes,industry_code,...,mod_short_line_score_change,mod_short_line_rank,mod_trend_score,mod_trend_score_change,mod_trend_rank,env_json_info,block_category_info,created_at,return,real_return
0,920,2024-09-02,低吸,低位N字低吸,002144.XSHE,宏达高科,2,,,,...,8.81,2,107.82,2.64,2,"{""9A0001"": {""realShortLineScore"": 23.04, ""real...","{""000027.BKDL"": {""categoryCode"": ""000027.BKDL""...",2025-02-15 17:42:06,-0.001159,-0.004159
1,947,2024-09-03,低吸,低位N字低吸,002428.XSHE,云南锗业,1,,,,...,2.71,1,108.22,-12.05,1,"{""9A0001"": {""realShortLineScore"": 20.72, ""real...","{""000028.BKDL"": {""categoryCode"": ""000028.BKDL""...",2025-02-15 17:43:09,-0.012490,-0.015490
2,1040,2024-09-05,低吸,低位N字低吸,600543.XSHG,莫高股份,3,,,,...,11.16,2,94.02,3.22,2,"{""9A0001"": {""realShortLineScore"": -12.1, ""real...","{""000003.BKDL"": {""categoryCode"": ""000003.BKDL""...",2025-02-15 17:45:19,0.001980,-0.001020
3,1086,2024-09-06,低吸,低位N字低吸,600156.XSHG,华升股份,2,,,,...,-0.44,2,99.63,-1.00,2,"{""9A0001"": {""realShortLineScore"": -22.6, ""real...","{""000027.BKDL"": {""categoryCode"": ""000027.BKDL""...",2025-02-15 17:46:23,-0.050000,-0.053000
4,1138,2024-09-09,低吸,低位N字低吸,002177.XSHE,御银股份,1,"000003.BKDL,000023.BKDL,000014.BKDL,000015.BKD...","885757.DDBK,885413.DDBK,885742.DDBK,885866.DDB...",980366.ZHBK,...,-15.32,5,95.43,-7.86,3,"{""9A0001"": {""realShortLineScore"": -19.29, ""rea...","{""000009.BKDL"": {""categoryCode"": ""000009.BKDL""...",2025-02-15 17:47:27,0.021407,0.018407
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,1317,2025-02-19,低吸,低位N字低吸,002786.XSHE,银宝山新,1,"000012.BKDL,000023.BKDL,000029.BKDL,000014.BKD...","885740.DDBK,885785.DDBK,886021.DDBK,885517.DDB...",980365.ZHBK,...,-7.99,76,-38.83,-5.50,52,"{""9A0001"": {""realShortLineScore"": 17.18, ""real...","{""000033.BKDL"": {""categoryCode"": ""000033.BKDL""...",2025-02-20 08:01:09,0.051456,0.048456
80,1378,2025-02-20,低吸,低位N字低吸,001298.XSHE,好上好,1,"000026.BKDL,000014.BKDL,000031.BKDL,000027.BKD...","885785.DDBK,885840.DDBK,886042.DDBK,885312.DDB...","980357.ZHBK,980345.ZHBK",...,18.63,61,-15.15,1.03,44,"{""9C0001"": {""realShortLineScore"": 38.94, ""real...","{""000033.BKDL"": {""categoryCode"": ""000033.BKDL""...",2025-02-20 08:02:03,0.000000,-0.003000
81,1708,2025-02-25,低吸,低位N字低吸,603206.XSHG,嘉环科技,1,"000012.BKDL,000003.BKDL,000035.BKDL,000011.BKD...","885919.DDBK,885461.DDBK,885956.DDBK,885556.DDB...","980753.ZHBK,980376.ZHBK",...,1.16,38,-3.11,-9.69,33,"{""9A0001"": {""realShortLineScore"": 38.02, ""real...","{""000033.BKDL"": {""categoryCode"": ""000033.BKDL""...",2025-02-26 08:01:18,0.019651,0.016651
82,1848,2025-02-26,低吸,低位N字低吸,603789.XSHG,星光农机,2,"000003.BKDL,000024.BKDL,000023.BKDL,000010.BKD...","885924.DDBK,885739.DDBK,885706.DDBK,885517.DDB...",980365.ZHBK,...,-31.83,42,-3.49,-6.46,40,"{""9A0001"": {""realShortLineScore"": 36.86, ""real...","{""000027.BKDL"": {""categoryCode"": ""000027.BKDL""...",2025-02-27 08:01:10,0.008333,0.005333


In [7]:

# pd.set_option('display.max_rows', None)
for k, df in m.items():
# combined_df = combined_df.drop(['env_json_info', 'block_category_info'], axis=1)
    df = df.drop(['block_category_info'], axis=1)
    # 将索引设置为 date_key 列
    df = df.set_index('date_key')
    df =  df[(df['open_price'] > 0) & (df['next_day_open_price'] > 0)]

    # 对索引进行排序
    df = df.sort_index()

    dwndx = df[df['strategy_name'] == strategy_name]
    
    dwndx['cumulative_returns'] = (1 + dwndx['real_return']).cumprod()

    print(k,': ', len(df))
    print(f"{k} return:")

    r = caculate_returns(dwndx, 'real_return', _print=True)
    # 打开一个文件以写入模式
    with open(f'{sub_strategy_name}-returns_result{months[0]}.txt', 'a') as f:
        f.write(f"{k} result: \n")
        # 遍历字典中的键值对
        for key, value in r.items():
            # 将键值对以 key: value 的格式写入文件，每个键值对占一行
            f.write(f"{key}: {value}\n")
        f.write('\n')
    
    print('-----------------------------------------------------')
    print('')


rank1 :  82
rank1 return:
最大回撤: -26.43%
夏普比率: 0.25
总收益率: 238.82%
年化收益率: 698.99%
波动率: 6.87%
总盈亏: 238.87%
成功次数: 48
失败次数: 34
总天数: 82
总交易次数: 82
交易频率: 100.00%
胜率: 58.54%
平均盈利: 5.37%
平均亏损: -3.44%
最大盈利: 27.04%
最大亏损: -11.28%
盈亏比: 1.56
凯利公式最佳仓位: 31.98%
-----------------------------------------------------

rank3 :  82
rank3 return:
最大回撤: -21.12%
夏普比率: 0.21
总收益率: 102.66%
年化收益率: 300.48%
波动率: 4.60%
总盈亏: 102.90%
成功次数: 44
失败次数: 38
总天数: 82
总交易次数: 82
交易频率: 100.00%
胜率: 53.66%
平均盈利: 4.02%
平均亏损: -2.57%
最大盈利: 16.77%
最大亏损: -11.98%
盈亏比: 1.56
凯利公式最佳仓位: 24.04%
-----------------------------------------------------

fx1 :  82
fx1 return:
最大回撤: -23.66%
夏普比率: 0.14
总收益率: 64.73%
年化收益率: 189.47%
波动率: 5.26%
总盈亏: 65.15%
成功次数: 45
失败次数: 37
总天数: 82
总交易次数: 82
交易频率: 100.00%
胜率: 54.88%
平均盈利: 3.97%
平均亏损: -3.19%
最大盈利: 19.59%
最大亏损: -9.71%
盈亏比: 1.25
凯利公式最佳仓位: 18.66%
-----------------------------------------------------

rank2 :  82
rank2 return:
最大回撤: -24.01%
夏普比率: 0.23
总收益率: 124.75%
年化收益率: 365.13%
波动率: 4.75%
总盈亏: 124.98%
成功次数: 

In [7]:
# fx1 :  81
# fx1 return:
# 最大回撤: -19.71%
# 夏普比率: 0.34
# 总收益率: 313.00%
# 年化收益率: 927.41%
# 波动率: 5.67%
# 总盈亏: 313.94%
# 成功次数: 53
# 失败次数: 28
# 总天数: 81
# 总交易次数: 81
# 交易频率: 100.00%
# 胜率: 65.43%
# 平均盈利: 4.26%
# 平均亏损: -2.53%
# 最大盈利: 22.71%
# 最大亏损: -9.60%
# 盈亏比: 1.69
# 凯利公式最佳仓位: 44.93%


# rank1 :  81
# rank1 return:
# 最大回撤: -19.36%
# 夏普比率: 0.28
# 总收益率: 244.39%
# 年化收益率: 724.12%
# 波动率: 6.00%
# 总盈亏: 245.33%
# 成功次数: 51
# 失败次数: 30
# 总天数: 81
# 总交易次数: 81
# 交易频率: 100.00%
# 胜率: 62.96%
# 平均盈利: 4.42%
# 平均亏损: -2.91%
# 最大盈利: 22.71%
# 最大亏损: -9.60%
# 盈亏比: 1.52
# 凯利公式最佳仓位: 38.59%



# rank3 :  81
# rank3 return:
# 最大回撤: -13.09%
# 夏普比率: 0.35
# 总收益率: 314.43%
# 年化收益率: 931.64%
# 波动率: 5.48%
# 总盈亏: 315.36%
# 成功次数: 54
# 失败次数: 27
# 总天数: 81
# 总交易次数: 81
# 交易频率: 100.00%
# 胜率: 66.67%
# 平均盈利: 4.25%
# 平均亏损: -2.77%
# 最大盈利: 22.71%
# 最大亏损: -13.09%
# 盈亏比: 1.53
# 凯利公式最佳仓位: 44.92%


In [ ]:
print("return:")
caculate_returns(dwndx, 'real_return', _print=True)
print('-----------------------------------------------------')

In [ ]:
plot(dwndx, 'cumulative_returns')

In [ ]:
dwndx1 = dwndx[['stock_code', 'stock_name', 'open_price', 'max_block_category_rank','max_block_code_rank','max_industry_code_rank','close_price', 'next_day_open_price', 'next_day_close_price', 'real_return', 'mod_short_line_rank', 'mod_name','mod_short_line_rank', 'mod_trend_rank', 'return',
       'real_return', 'cumulative_returns']]
dwndx1

In [ ]:
plot(df, 'first_cumulative_returns')

In [ ]:
plot(df, 'p_first_cumulative_returns')

In [ ]:
plot(df, 'hd_cumulative_returns')

In [ ]:
plot(df, 'hd_p_cumulative_returns')

In [ ]:
len(df_g_zero)

In [ ]:
len(df)

In [ ]:
len(df)/len(df_g_zero)

In [ ]:
df_after_date = df.loc[df.index >= '2025-01-01']
df_after_date['cumulative_returns'] = (1 + df_after_date['return']).cumprod()
df_after_date['p_return'] = df_after_date['return'] * df_after_date['position']
df_after_date['p_cumulative_returns'] = (1 + df_after_date['p_return']).cumprod()
df_after_date['first_cumulative_returns'] = (1 + df_after_date['first_return']).cumprod()
df_after_date['p_first_return'] = df_after_date['first_return'] * df_after_date['position']
df_after_date['p_first_cumulative_returns'] = (1 + df_after_date['p_first_return']).cumprod()
df_after_date_g_zero = df_after_date[df_after_date['codes_num'] > 0]

In [ ]:
print("return:")
caculate_returns(df_after_date_g_zero, 'return', True)
print('-------------------------------------------------------------------')

In [ ]:
print("p return:")
caculate_returns(df_after_date_g_zero, 'p_return', True)
print("---------------------------------------------------------------------")


In [ ]:
print("first return:")
caculate_returns(df_after_date_g_zero, 'first_return', True)
print("----------------------------------------------------------------------")


In [ ]:
print("p first_return:")
caculate_returns(df_after_date_g_zero, 'p_first_return', True)
print("-----------------------------------------------------------------------")

In [ ]:
plot(df_after_date, 'cumulative_returns')

In [ ]:
plot(df_after_date, 'p_cumulative_returns')

In [ ]:
plot(df_after_date, 'first_cumulative_returns')

In [ ]:
plot(df_after_date, 'p_first_cumulative_returns')

In [ ]:
0.008906*0.3

In [ ]:
3.61*0.3

In [ ]:
df_g_zero

# 不按凯利公式调仓
## return
- 最大回撤: -20.73%
- 夏普比率: 0.09
- 总收益率: 14.75%
- 波动率: 5.90%
- 最大收益: 16.11%
- 总盈亏: 11.40%
- 成功次数: 19
- 胜率: 51.35%
- 平均盈利: 5.08%
- 平均亏损: -4.25%
- 最大盈利: 16.11%
- 最大亏损: -8.12%
- 盈亏比: 1.20
- 凯利公式最佳仓位: 10.67%

## p return
- 最大回撤: -13.76%
- 夏普比率: 0.19
- 总收益率: 28.26%
- 波动率: 3.91%
- 最大收益: 14.37%
- 总盈亏: 25.16%
- 成功次数: 19
- 胜率: 51.35%
- 平均盈利: 3.46%
- 平均亏损: -2.12%
- 最大盈利: 14.37%
- 最大亏损: -4.99%
- 盈亏比: 1.64
- 凯利公式最佳仓位: 21.61%

## first return
- 最大回撤: -13.93%
- 夏普比率: 0.12
- 总收益率: 23.89%
- 波动率: 5.88%
- 最大收益: 16.11%
- 总盈亏: 20.55%
- 成功次数: 20
- 胜率: 54.05%
- 平均盈利: 4.98%
- 平均亏损: -4.23%
- 最大盈利: 16.11%
- 最大亏损: -9.67%
- 盈亏比: 1.18
- 凯利公式最佳仓位: 15.04%

## p first_return
- 最大回撤: -13.74%
- 夏普比率: 0.20
- 总收益率: 31.27%
- 波动率: 3.85%
- 最大收益: 14.37%
- 总盈亏: 28.17%
- 成功次数: 20
- 胜率: 54.05%
- 平均盈利: 3.30%
- 平均亏损: -2.12%
- 最大盈利: 14.37%
- 最大亏损: -4.99%
- 盈亏比: 1.55
- 凯利公式最佳仓位: 24.50%

## p3 return
- 最大回撤: -6.54%
- 夏普比率: 0.09
- 总收益率: 5.59%
- 波动率: 1.77%
- 最大收益: 4.83%
- 总盈亏: 4.58%
- 成功次数: 19
- 胜率: 51.35%
- 平均盈利: 1.52%
- 平均亏损: -1.27%
- 最大盈利: 4.83%
- 最大亏损: -2.44%
- 盈亏比: 1.20
- 凯利公式最佳仓位: 10.67%

## p3 first return
- 最大回撤: -4.28%
- 夏普比率: 0.12
- 总收益率: 8.05%
- 波动率: 1.76%
- 最大收益: 4.83%
- 总盈亏: 7.04%
- 成功次数: 20
- 胜率: 54.05%
- 平均盈利: 1.50%
- 平均亏损: -1.27%
- 最大盈利: 4.83%
- 最大亏损: -2.90%
- 盈亏比: 1.18
- 凯利公式最佳仓位: 15.04%


<!-- return:
最大回撤: -20.73%
夏普比率: 0.04
总收益率: 1.99%
波动率: 5.35%
最大收益: 10.37%
总盈亏: 5.31%
None
p return:
最大回撤: -7.64%
夏普比率: 0.21
总收益率: 14.38%
波动率: 3.52%
最大收益: 10.37%
总盈亏: 17.71%
None
first return:
最大回撤: -12.83%
夏普比率: 0.13
总收益率: 12.16%
波动率: 5.23%
最大收益: 10.37%
总盈亏: 15.49%
None
p first_return:
最大回撤: -4.61%
夏普比率: 0.26
总收益率: 18.58%
波动率: 3.44%
最大收益: 10.37%
总盈亏: 21.91%
None -->


# 按照凯利公式调仓
## return
- 最大回撤: -20.73%
- 夏普比率: 0.09
- 总收益率: 14.75%
- 波动率: 5.90%
- 最大收益: 16.11%
- 总盈亏: 11.40%
- 成功次数: 19
- 总次数: 37
- 胜率: 51.35%
- 平均盈利: 5.08%
- 平均亏损: -4.25%
- 最大盈利: 16.11%
- 最大亏损: -8.12%
- 盈亏比: 1.20
- 凯利公式最佳仓位: 10.67%

## p return
- 最大回撤: -13.76%
- 夏普比率: 0.19
- 总收益率: 28.26%
- 波动率: 3.91%
- 最大收益: 14.37%
- 总盈亏: 25.16%
- 成功次数: 19
- 总次数: 37
- 胜率: 51.35%
- 平均盈利: 3.46%
- 平均亏损: -2.12%
- 最大盈利: 14.37%
- 最大亏损: -4.99%
- 盈亏比: 1.64
- 凯利公式最佳仓位: 21.61%

## first return
- 最大回撤: -13.93%
- 夏普比率: 0.12
- 总收益率: 23.89%
- 波动率: 5.88%
- 最大收益: 16.11%
- 总盈亏: 20.55%
- 成功次数: 20
- 总次数: 37
- 胜率: 54.05%
- 平均盈利: 4.98%
- 平均亏损: -4.23%
- 最大盈利: 16.11%
- 最大亏损: -9.67%
- 盈亏比: 1.18
- 凯利公式最佳仓位: 15.04%

## p first_return
- 最大回撤: -13.74%
- 夏普比率: 0.20
- 总收益率: 31.27%
- 波动率: 3.85%
- 最大收益: 14.37%
- 总盈亏: 28.17%
- 成功次数: 20
- 总次数: 37
- 胜率: 54.05%
- 平均盈利: 3.30%
- 平均亏损: -2.12%
- 最大盈利: 14.37%
- 最大亏损: -4.99%
- 盈亏比: 1.55
- 凯利公式最佳仓位: 24.50%

## p3 return
- 最大回撤: -2.21%
- 夏普比率: 0.09
- 总收益率: 1.96%
- 波动率: 0.59%
- 最大收益: 1.61%
- 总盈亏: 1.62%
- 成功次数: 19
- 总次数: 37
- 胜率: 51.35%
- 平均盈利: 0.51%
- 平均亏损: -0.42%
- 最大盈利: 1.61%
- 最大亏损: -0.81%
- 盈亏比: 1.20
- 凯利公式最佳仓位: 10.67%

## p3 first return
- 最大回撤: -2.15%
- 夏普比率: 0.12
- 总收益率: 4.10%
- 波动率: 0.88%
- 最大收益: 2.42%
- 总盈亏: 3.59%
- 成功次数: 20
- 总次数: 37
- 胜率: 54.05%
- 平均盈利: 0.75%
- 平均亏损: -0.63%
- 最大盈利: 2.42%
- 最大亏损: -1.45%
- 盈亏比: 1.18
- 凯利公式最佳仓位: 15.04%


# 10月15号后
## return
- 最大回撤: -20.73%
- 夏普比率: 0.04
- 总收益率: 1.99%
- 波动率: 5.35%
- 最大收益: 10.37%
- 总盈亏: 5.31%
- 成功次数: 10
- 总次数: 19
- 胜率: 52.63%
- 平均盈利: 4.50%
- 平均亏损: -4.48%
- 最大盈利: 10.37%
- 最大亏损: -8.12%
- 盈亏比: 1.00
- 凯利公式最佳仓位: 5.48%

## p return
- 最大回撤: -7.64%
- 夏普比率: 0.21
- 总收益率: 14.38%
- 波动率: 3.52%
- 最大收益: 10.37%
- 总盈亏: 17.71%
- 成功次数: 10
- 总次数: 19
- 胜率: 52.63%
- 平均盈利: 3.30%
- 平均亏损: -2.04%
- 最大盈利: 10.37%
- 最大亏损: -4.61%
- 盈亏比: 1.62
- 凯利公式最佳仓位: 23.34%

## first return
- 最大回撤: -12.83%
- 夏普比率: 0.13
- 总收益率: 12.16%
- 波动率: 5.23%
- 最大收益: 10.37%
- 总盈亏: 15.49%
- 成功次数: 11
- 总次数: 19
- 胜率: 57.89%
- 平均盈利: 4.49%
- 平均亏损: -4.41%
- 最大盈利: 10.37%
- 最大亏损: -8.12%
- 盈亏比: 1.02
- 凯利公式最佳仓位: 16.54%

## p first_return
- 最大回撤: -4.61%
- 夏普比率: 0.26
- 总收益率: 18.58%
- 波动率: 3.44%
- 最大收益: 10.37%
- 总盈亏: 21.91%
- 成功次数: 11
- 总次数: 19
- 胜率: 57.89%
- 平均盈利: 3.13%
- 平均亏损: -2.03%
- 最大盈利: 10.37%
- 最大亏损: -4.61%
- 盈亏比: 1.54
- 凯利公式最佳仓位: 30.60%
